# 설정

In [2]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "Boosting"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 부스팅

<b>부스팅</b><sup>boosting</sup>(원래는 <b>가설 부스팅</b><sup>hypothesis boosting</sup>이라 불렀다)은 <b>약한 학습기</b><sup>weak learner</sup>라고도 하는 매우 간단한 분류기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법을 말한다. 이 약한 학습기는 랜덤 추측보다 조금 성능이 좋을 뿐이며 전형적인 예로는 깊이가 1인 결정 트리를 들 수 있다. 부스팅의 핵심 아이디어는 분류하기 어려운 훈련 샘플에 초점을 맞추는 것이다. 즉, 잘못 분류된 훈련 샘플을 그다음 약한 학습기가 학습하여 앙상블 성능을 향상시킨다.

이어지는 절에서 일반적인 부스팅 알고리즘의 절차를 소개하고 가장 인기 있는 구현인 <b>에이다부스트</b><sup>AdaBoost</sup>(adaptive boosting의 줄임말)를 소개해보겠다.

## 부스팅 작동 원리

배깅과는 달리 부스팅의 초창기 방법은 중복을 허용하지 않고 훈련 세트에서 랜덤 샘플을 추출하여 부분 집합을 구성한다. 원본 부스팅 과정은 다음 네 개의 주요 단계로 요약할 수 있다.

<ol>
    <li>훈련 세트 $D$에서 중복을 허용하지 않고 랜덤한 부분 집합 $d_1$을 뽑아 약한 학습기 $C_1$을 훈련한다.</li>
    <li>훈련 세트에서 중복을 허용하지 않고 두 번째 랜덤한 훈련 부분 집합 $d_2$를 뽑고 이전에 잘못 분류된 샘플의 50%를 더해서 약한 학습기 $C_2$를 훈련한다.</li>
    <li>훈련 세트 $D$에서 $C_1$과 $C_2$에서 잘못 분류한 훈련 샘플 $d_3$를 찾아 세 번째 약한 학습기인 $C_3$를 훈련한다.</li>
    <li>약한 학습기 $C_1$, $C_2$, $C_3$를 다수결 투표로 연결한다.</li>
</ol>

레오 브레이만이 언급한 것처럼 부스팅은 배깅 모델에 비해 분산은 물론 편향도 감소시킬 수 있다. 실제로는 에이다부스트 같은 부스팅 알고리즘이 분산이 높다고 알려져 있다. 즉, 훈련 데이터에 과대적합되는 경향이 있다.

여기서 언급한 원본 부스팅 방법과는 다르게 에이다부스트는 약한 학습기를 훈련할 때 훈련 세트 전체를 사용한다. 훈련 샘플은 반복마다 가중치가 다시 부여되며 이 앙상블은 이전 학습기의 실수를 학습하는 강력한 분류기를 만든다. 에이다부스트 알고리즘을 구체적으로 깊게 알아보기 전에 [그림 1]로 에이다부스트의 기본 개념을 좀 더 잘 이해해 보겠다.

<b>그림 1</b> 에이다부스트
<div style="text-align:center;">
    <img src="./images/Boosting/에이다부스트.png">
</div>

이 에이다부스트 예시를 하나씩 따라가 보자. 그림 1-1은 이진 분류를 위한 훈련 세트를 보여준다. 여기서 모든 샘플은 동일한 가중치를 가진다. 이 훈련 세트를 바탕으로 깊이가 1인 결정 트리(파선)를 훈련하여 샘플을 두 개의 클래스(삼각형과 원)로 나눈다. 물론 가능한 비용 함수(또는 결정 트리 앙상블일 경우 불슨도 점수)를 최소화하는 트리를 훈련한다.

다음 단계 그림 1-2에서 이전에 잘못 분류된 샘플 두 개(원)에 큰 가중치를 부여한다. 또 옳게 분류된 샘플의 가중치는 낮춘다. 다음 결정 트리는 가장 큰 가중치를 가진 훈련 샘플에 더 집중할 것이다. 아마도 이런 훈련 샘플은 분류하기 어려운 샘플일 것이다. 그림 1-2에 있는 약한 학습기는 세 개의 원 모양 샘플을 잘못 분류한다. 그림 1-3에서 이 샘플들에 큰 가중치가 부여된다.

이 에이다부스트 앙상블이 세 번의 부스팅 단계만을 가진다고 가정하면 그림 1-4에서처럼 서로 다른 가중치가 부여된 훈련 세트에서 훈련된 세 개의 약한 학습기를 다수결 투표 방식으로 합친다.

이제 의사 코드를 사용하여 자세히 알고리즘을 살펴보자. 여기서 곱셈 기호($\times$)는 원소별 곱셈을 말하고 점 기호($\cdot$)는 두 벡터 사이의 점곱을 의미한다.

<ol>
    <li>가중치 벡터 $\mathrm{w}$를 동일한 가중치로 설정한다. $\sum_i\mathrm{w}_i = 1$</li>
    <li>$m$번 부스팅 반복의 $j$번째에서 다음을 수행한다.
        <ol>
            <li>가중치가 부여된 약한 학습기를 훈련한다. $C_j = \text{train}(\mathrm{X, y, w})$</li>
            <li>클래스 레이블을 예측한다. $\hat{y} = \text{predict}(C_j, \mathrm{X})$</li>
            <li>가중치가 적용된 에러율을 계산한다. $\varepsilon = \mathrm{w}\cdot(\hat{y}\neq y)$</li>
            <li>학습기 가중치를 계산한다. $\alpha_j = 0.5log\frac{1-\varepsilon}{\varepsilon}$</li>
            <li>가중치를 업데이트한다. $\mathrm{w}:=\mathrm{w}\, \times\,\text{exp}(-\alpha_j\,\times\,\hat{y}\,\times\,y)$</li>
            <li>합이 1이 되도록 가중치를 정규화한다. $\mathrm{w}:=\frac{\mathrm{w}}{\sum_i\mathrm{w}_i}$</li>
        </ol></li>
    <li>최종 예측을 계산한다. $\hat{y} = (\sum_{j=1}^m(\alpha_j\,\times\,\text{predict}(C_j, \mathrm{X})) > 0)$

단계 2-C에서 $(\hat{y}\neq y)$ 표현은 1 또는 0으로 구성된 이진 벡터를 의미한다. 예측이 잘못되면 1이고 그렇지 않으면 0이다.

에이다부스트 알고리즘이 간단해 보이지만, [그림 2]의 표와 같이 열 개의 훈련 샘플로 구성된 훈련 세트를 사용하여 구체적인 예제를 살펴보겠다.

<b>그림 2</b> 열 개의 훈련 샘플로 구성된 훈련 세트
<div style="text-align:center;">
    <img src="./images/Boosting/열 개의 훈련 샘플.jpg">
</div>